# Faster R-CNN Kitti Inference in Amazon SageMaker

This notebook is a step-by-step tutorial on [Faster R-CNN](https://arxiv.org/abs/1506.01497) model inference using [Amazon SageMaker model deployment hosting service](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html). The model is trained on [Kitti](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d) 2-D object dataset.

To get started, we initialize an Amazon execution role and initialize a `boto3` session to find our AWS region name.

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

session = boto3.session.Session()
aws_region = session.region_name
print(f'AWS region:{aws_region}')

## Build and push SageMaker serving image

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon ECR service. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to ECR service. 

Below, we will build an image for [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) implementation and push it to Amazon ECR.

### Build and Push TensorPack Faster-RCNN/Mask-RCNN Serving Container Image

Use ```./container-kitti-serving/build_tools/build_and_push.sh``` script to build and push the [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) <b>serving</b> container image to Amazon ECR. 

In [ ]:
!cat ./container-kitti-serving/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container-kitti-serving/build_tools/build_and_push.sh {aws_region}

Set ```tensorpack_image``` below to Amazon ECR URI of the <b>serving</b> image you pushed above.

In [ ]:
tensorpack_image = #<amazon-ecr-uri>

Next, we set ```serving_image``` to  the `tensorpack_image`.

In [ ]:
serving_image = tensorpack_image 
print(f'serving image: {serving_image}')

## Create Amazon SageMaker Session 
Next, we create a SageMaker session.

In [ ]:
sagemaker_session = sagemaker.session.Session(boto_session=session)

## Define Amazon SageMaker Model
Next, we define an Amazon SageMaker Model that defines the deployed model we will serve from an Amazon SageMaker Endpoint. 

In [ ]:
model_name= 'faster-rcnn-kitti-model-1'# Name of the model

Next we set the `s3_model_url` to the trained model S3 URL. 

The environment variable `RESNET_ARCH` must be set to `resnet50` or `resnet101`. For any, configuration variable, prefix the variable with `CONFIG__` and replace all `.` with `__`. For example, if the configuration variable is `RPN.ANCHOR_SIZES`, set the environment variable `CONFIG__RPN__ANCHOR_SIZES`, as shown below.

In [ ]:
s3_model_url =  # Trained Model Amazon S3 URI in the format s3://<your path>/model.tar.gz
serving_container_def = {
    'Image': serving_image,
    'ModelDataUrl': s3_model_url,
    'Mode': 'SingleModel',
    'Environment': { 'SM_MODEL_DIR' : '/opt/ml/model',
                     'RESNET_ARCH': 'resnet50', 
                     'CONFIG__RPN__ANCHOR_SIZES': '(32, 64, 128, 256, 512)'
                   }}

create_model_response = sagemaker_session.create_model(name=model_name, 
                                                       role=role, 
                                                       container_defs=serving_container_def)

print(create_model_response)

 Next, we set the name of the Amaozn SageMaker hosted service endpoint configuration.

In [ ]:
endpoint_config_name=f'{model_name}-endpoint-config'
print(endpoint_config_name)

Next, we create the Amazon SageMaker hosted service endpoint configuration that uses one instance of `ml.p3.2xlarge` to serve the model.

In [ ]:
epc = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name, 
    model_name=model_name, 
    initial_instance_count=1, 
    instance_type='ml.p3.2xlarge')
print(epc)

Next we specify the Amazon SageMaker endpoint name for the endpoint used to serve the model.

In [ ]:
endpoint_name=f'{model_name}-endpoint'
print(endpoint_name)

Next, we create the Amazon SageMaker endpoint using the endpoint configuration we created above.

In [ ]:
ep=sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=endpoint_config_name, wait=True)
print(ep)

Now that the Amazon SageMaker endpoint is in service, we will use the endpoint to do inference for test images. 

Next, we download Kitti 2-D object dataset testing images from your S3 bucket. It is assumed that you have staged [Kitti](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d) 2-D object dataset testing images in your bucket under the prefix `/faster-rcnn-kitti/testing/`. The estimated time for downloading the testing images from your S3 bucket is approximately 1 minute.

In [ ]:
%%time
s3_bucket= # <your-s3-bucket-name>
! aws s3 cp --quiet --recursive s3://{s3_bucket}/faster-rcnn-kitti/testing/ /tmp/testing/

Below, we will use the Kitti 2-D object dataset testing images to test our deployed Faster R-CNN model trained on Kitti 2-D object dataset. However, in order to visualize the detection results, we need to define some helper functions.

## Visualization Helper Functions
Next, we define a helper function for generating random colors for visualizing detection results.

In [ ]:
import colorsys
import random

def random_colors(N, bright=False):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

Next, we define a helper function to show the applied detection results.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

def show_detection_results(img=None,
                            annotations=None):
    """
    img: image numpy array
    annotations: annotations array for image where each annotation is in COCO format
    """
    num_annotations = len(annotations)
    colors = random_colors(num_annotations)
    
    fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
    
    for i, a in enumerate(annotations):
        bbox = a['bbox']
        category_id = a['category_id']
        category_name = a['category_name']
    
        # select color from random colors
        color = colors[i]

        # Show bounding box
        bbox_x, bbox_y, bbox_w, bbox_h = bbox

        box_patch = patches.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, 
                        linewidth=1,
                        alpha=0.7, linestyle="dashed",
                        edgecolor=color, facecolor='none')
        ax.add_patch(box_patch)
        label = f'{category_name}:{category_id}'
        ax.text(bbox_x, bbox_y + 8, label,
                color='w', size=11, backgroundcolor="none")

    
    ax.imshow(img.astype(int))
    plt.show()

## Visualize Detection Results
Next, we select a random image from Kitti 2-D object dataset testing images. After you are done visualizing the detection results for this image, you can come back to the cell below and select your next random image to test.

In [ ]:
import os
import random

testing_dir=os.path.join('/tmp', "testing")
img_id=random.choice(os.listdir(testing_dir))
img_local_path = os.path.join(testing_dir,img_id)
print(img_local_path)

Next, we read the image and convert it from BGR color to RGB color format.

In [ ]:
import cv2

img=cv2.imread(img_local_path, cv2.IMREAD_COLOR)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Next, we show the image that we randomly selected.

In [ ]:
fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))
plt.show()

Next, we invoke the Amazon SageMaker Endpoint to detect objects in the test image that we randomly selected.

This REST API endpoint only accepts HTTP POST requests with `ContentType` set to `application/json`. The content of the POST request must conform to following JSON schema:

`{ 
    "img_id": "YourImageId", 
    "img_data": "Base64 encoded image file content, encoded as utf-8 string" 
 }`

The response of the POST request conforms to following JSON schema:

`{ 
    "annotations": [ 
                    {
                        "bbox": [X, Y, width, height], 
                        "category_id": "class id", 
                        "category_name": "class name", 
                        "segmentation": { "counts": [ run-length-encoding, ], "size": [height, width]} 
                     },
                   ]
 }`

In [ ]:
import boto3 
import base64
import json

client = boto3.client('sagemaker-runtime')

with open(img_local_path, "rb") as image_file:
    img_data = base64.b64encode(image_file.read())
    data = {"img_id": img_id}
    data["img_data"] = img_data.decode('utf-8')
    body=json.dumps(data).encode('utf-8')
    
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=body)
body=response['Body'].read()
msg=body.decode('utf-8')
data=json.loads(msg)
assert data is not None

The response from the endpoint includes annotations for the detected objects in COCO annotations format. 

Next, we aplly all the detection results to the image. 

In [ ]:
annotations = data['annotations']
show_detection_results(img, annotations)

## Delete SageMaker Endpoint, Endpoint Config and Model
If you are done testing, delete the deployed Amazon SageMaker endpoint, endpoint config, and the model below. The trained model in S3. bucket is not deleted. If you are not done testing, go back to the section <b>Visualize Detection Results</b> and select another test image.

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=endpoint_name)
sagemaker_session.delete_endpoint_config(endpoint_config_name=endpoint_config_name)
sagemaker_session.delete_model(model_name=model_name)